<a href="https://colab.research.google.com/github/chanapacha/Pantip-WebScraping/blob/main/Pantip_Webscraping_Using_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping โดยใช้ pantipScraper

https://github.com/Bankde/pantip_scraper/blob/master/pantipScraper.py

# Install library

In [1]:
!pip install requests lxml

# Write pantipScraper file

In [2]:
%%file pantipScraper.py

#!/opt/local/bin/python

#############################################################
#															#
#  Created by DarkDrag0nite									#
#															#
#  To use: python pantipScraper.py <start_topic_id>			#
#  Example: python pantipScraper.py 35000000				#
#															#
#############################################################


from lxml import html
import json
import requests
from requests.exceptions import ConnectionError, ReadTimeout
import time
import random
import os, sys
import re
import codecs

udg_thaiEncode = 'utf-8-sig'
udg_header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:42.0) Gecko/20100101 Firefox/42.0'}
udg_header_comment = {
	'Host': 'pantip.com',
	'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:42.0) Gecko/20100101 Firefox/42.0',
	'Accept': 'application/json, text/javascript, */*; q=0.01',
	'Accept-Language': 'en-US,en;q=0.5',
	'Accept-Encoding': 'gzip, deflate',
	'X-Requested-With': 'XMLHttpRequest',
	'Connection': 'keep-alive'
}
udg_storage_dir = "pantip_storage"

# GLOBAL MODE # // This shouldn't be global but global is easier for faster develop
disableCommentFeature = False

class ReturnData:

	def __init__(self, status, data):
		self.status = status
		self.data = data

	def getStatus(self):
		return self.status

	def getData(self):
		return self.data

class PantipCrawler:

	def __init__(self, tid):
		self.tid = tid
		self.topic = ''

	def crawl(self):
		while True:
			try:
				# Get Main Topic
				functionData = Topic.get_topic_from_link(self.tid)
				if functionData.getStatus() == False:
					return functionData
				else:
					self.topic = functionData.getData()

				# Get Comments
				functionData = Comment.get_comments_from_link(self.tid)
				if functionData.getStatus == False:
					return functionData
				else:
					comments = functionData.getData()

				# Get Comment count
				if comments:
					commentCount = len(comments)
				else:
					commentCount = 0

				self.topic.commentCount = commentCount
				self.topic.comments = comments

				rData = ReturnData(True, "Success: Crawling page %s"%(self.tid))
				return rData
			except ConnectionError as e:
				print("Connection error: Program will try again in 10s. Ctrl+c to quit")
				time.sleep(10)
			except ReadTimeout as e:
				print("Request timeout: Program will try again in 10s. Ctrl+c to quit")
				time.sleep(10)

	def __str__(self):
		return str(self.topic)

	def toDict(self):
		tempDict = self.topic.toDict()
		return tempDict

	def toJson(self):
		return json.dumps(self.toDict(), ensure_ascii=False)

class Topic:
	def __init__(self, tid, name, author, author_id, story, likeCount, emoCount, emotions, tags, time, commentCount=0, comments=[]):
		self.tid = tid
		self.name = name
		self.author = author
		self.author_id = author_id
		self.story = story
		self.likeCount = likeCount
		self.emoCount = emoCount
		self.emotions = emotions
		self.tagList = tags
		self.dateTime = time
		self.commentCount = 0
		self.comments = []

	@staticmethod
	def get_topic_from_link(tid):
		global udg_header
		index = 0
		while(index < 4):
			start_page = requests.get("http://pantip.com/topic/%s"%(tid),
				headers=udg_header_comment)
			if (start_page.reason == 'OK'):
				break
			else:
				index = index + 1
			if index == 4:
				rData = ReturnData(False, "Cannot open page %s: "%(tid) + start_page.reason)
				return rData

		start_page.encoding = udg_thaiEncode
		tree = html.fromstring(start_page.text)

		tmp = tree.xpath('//div[starts-with(@class,"callback-status")]')
		if tmp and tmp[0].text_content():
			if not tree.xpath('//h2[@class="display-post-title"]/text()'):
				rData = ReturnData(False, tree.xpath('//div[starts-with(@class,"callback-status")]')[0].text_content().strip())
				return rData

		name = tree.xpath('//h2[@class="display-post-title"]/text()')[0]
		author = tree.xpath('//a[@class="display-post-name owner"]/text()')[0]
		author_id = tree.xpath('//a[@class="display-post-name owner"]/@id')[0]
		story = tree.xpath('//div[@class="display-post-story"]')[0].text_content()
		likeCount = tree.xpath('//span[starts-with(@class,"like-score")]/text()')[0]
		emoCount = tree.xpath('//span[starts-with(@class,"emotion-score")]/text()')[0]
		allEmos = tree.xpath('//span[@class="emotion-choice-score"]/text()')
		tags = tree.xpath('//div[@class="display-post-tag-wrapper"]/a[@class="tag-item"]/text()')
		dateTime = tree.xpath('//abbr[@class="timeago"]/@data-utime')[0]

		emotions = Emotion(allEmos[0], allEmos[1], allEmos[2], allEmos[3], allEmos[4], allEmos[5])
		topic = Topic(tid, name, author, author_id, story, likeCount, emoCount, emotions, tags, dateTime)
		rData = ReturnData(True, topic)
		return rData

	def toDict(self):
		return {
			'tid': self.tid,
			'name': self.name,
			'author' : self.author,
			'author_id' : self.author_id,
			'story' : self.story,
			'likeCount' : self.likeCount,
			'emoCount' : self.emoCount,
			'emotions' : self.emotions.toDict(),
			'tagList' : self.tagList,
			'dateTime' : self.dateTime,
			'commentCount' : self.commentCount,
			'comments' : [comment.toDict() for comment in self.comments]
		}

	def toJson(self):
		return json.dumps(self.toDict(), ensure_ascii=False)

	def __str__(self):
		return ("Name: " + self.name.encode(udg_thaiEncode) +
			"\r\nAuthor: " + self.author.encode(udg_thaiEncode) +
			"\r\nText: " + self.story.encode(udg_thaiEncode) + "\r\n"
			"\r\nLike Count: %s\r\nEmotion Count: %s"%(self.likeCount, self.emoCount) +
			"\r\nTag-item: " + ",".join(self.tagList).encode(udg_thaiEncode) +
			"\r\nDatetime: " + self.dateTime +
			"\r\nCommentCount: " + self.commentCount)

class Comment:
	def __init__(self, num, user_id, user_name, replyCount, replies, message, emotions, likeCount, dateTime):
		self.num = num
		self.user_id = user_id
		self.user_name = user_name
		self.replyCount = replyCount
		self.replies = replies
		self.message = message
		self.emotions = emotions
		self.likeCount = likeCount
		self.dateTime = dateTime

	@staticmethod
	def get_comments_from_link(tid):
		global udg_header_comment
		global disableCommentFeature

		udg_header_comment['Referer'] = "http://pantip.com/topic/%s"%(tid)
		comments = []
		index = 0
		while(index < 4):
			random_time = random.random()
			comment_response = requests.get("http://pantip.com/forum/topic/render_comments?tid=%s&param=&type=3&time=%s"%(tid, random_time),
					headers=udg_header_comment)
			if (comment_response.reason == 'OK'):
				break
			else:
				index = index + 1
			if index == 4:
				rData = ReturnData(False, "Cannot get comments %s: "%(tid) + start_page.reason)
				return rData

		comment_response.encoding = udg_thaiEncode
		comment_response_json = comment_response.json()

		if comment_response_json and 'count' not in comment_response_json:
			commentCount = 0
		else:
			commentCount = comment_response_json['count']

		if not disableCommentFeature and not commentCount == 0:
			for c in comment_response_json['comments']:
				comments.append(Comment.convertPantip2Python(c))

			if int(commentCount) > 100:
				pageIndex = 2
				while commentCount > (pageIndex-1)*100:
					index = 0
					while index < 4:
						random_time = random.random()
						comment_response = requests.get("http://pantip.com/forum/topic/render_comments?tid=%s&param=page%s&type=4&page=%s&parent=2&expand=1&time=%s"%(tid, pageIndex, pageIndex, random_time),
								headers=udg_header_comment)
						if (comment_response.reason == 'OK'):
							break
						else:
							index = index + 1
						if index == 4:
							rData = ReturnData(False, "Cannot get comments %s: "%(tid) + start_page.reason)
							return rData

					comment_response.encoding = udg_thaiEncode
					comment_response_json = comment_response.json()
					for c in comment_response_json['comments']:
						comments.append(Comment.convertPantip2Python(c))
					pageIndex += 1
		else:
			comments = {}

		rData = ReturnData(True, comments)
		return rData

	@staticmethod
	def convertPantip2Python(comment):
		# Coming soon
		replies = []
		emotions = Emotion.convertPantip2Python(comment['emotion'])
		return Comment(comment['comment_no'],
			comment['user']['mid'],
			comment['user']['name'],
			comment['reply_count'],
			replies,
			comment['message'],
			emotions,
			comment['point'],
			comment['data_utime'])

	def toDict(self):
		return {
			'num' : self.num,
			'user_id' : self.user_id,
			'user_name' : self.user_name,
			# I'm still thinking how should I handel the replies
			'replyCount' : self.replyCount,
			'replies' : self.replies,
			'message' : self.message,
			'emotions' : self.emotions.toDict(),
			'likeCount' : self.likeCount,
			'dateTime' : self.dateTime
		}

	def toJson(self):
		return json.dumps(self.toDict(), ensure_ascii=False)


class Emotion:
	def __init__(self, like=0, laugh=0, love=0, impress=0, scary=0, surprised=0):
		self.like = like
		self.laugh = laugh
		self.love = love
		self.impress = impress
		self.scary = scary
		self.surprised = surprised

	@staticmethod
	def convertPantip2Python(emotions):
		return Emotion(emotions['like']['count'],
			emotions['laugh']['count'],
			emotions['love']['count'],
			emotions['impress']['count'],
			emotions['scary']['count'],
			emotions['surprised']['count'])

	def toDict(self):
		return {
			'like' : self.like,
			'laugh' : self.laugh,
			'love' : self.love,
			'impress' : self.impress,
			'scary' : self.scary,
			'surprised' : self.surprised
		}

	def toJson(self):
		return json.dumps(self.toDict(), ensure_ascii=False)


def modeRoom(submode):
	random_time = random.random()
	topicList_response = requests.post("http://pantip.com/forum/topic/ajax_json_all_topic_info_loadmore?t=%s"%(random_time),
				data = {'last_id_current_page' : "35100000",
						'dataSend[room]' : "food",
						'dataSend[topic_type][type]' : "0",
						'dataSend[topic_type][default_type]' : "1",
						'thumbnailview' : "false",
						'current_page': "1"
						},
				headers=udg_header_comment)

	topicList = topicList_response.json()
	for topic in topicList['item']['topic']:
		print(topic['_id']),

def modeBruteID(submode):
	pageID = submode['pageID']
	endID = submode['endID']

	storage_file = 0
	f = None
	indexFile = open(udg_storage_dir + "/indexFile.txt", "w+")
	errorFile = open(udg_storage_dir + "/errorFile.txt", "a+")
	while (True):
		temp_file = str(int(pageID / 1000))
		if temp_file != storage_file:
			storage_file = temp_file
			f.close() if f else None
			f = open(udg_storage_dir + "/ptopic" + storage_file, "ab+")
		crawler = PantipCrawler(str(pageID))
		functionData = crawler.crawl()
		if functionData.getStatus() == True:
			f.write(crawler.toJson().encode(udg_thaiEncode))
			f.write(b"\n")
			indexFile.seek(0,0)
			indexFile.write("Done: %s\n"%(pageID))
			print(functionData.getData())
		else:
			errorFile.write("Failed: Crawling page %s: "%(pageID) + functionData.getData() + "\n")
			print("Failed: Crawling page %s: "%(pageID) + functionData.getData())
		pageID = pageID + 1
		if pageID > endID:
			break
		time.sleep(3)

def helpMode():
	print("""
== Quick use ==
To get a topic: python pantipScraper.py <topic_id>
Start from: python pantipScraper.py -start <topic_id>
End at: python pantipScraper.py -start <topic_id> -end <topic_id>

== Mode ==
\t-b\t\tbrute topic ID
\t-r <room>\tbrute from selected pantip room
\t-c\t\tcontinue from previous work
== Submode ==
\t-tid <id>\t get data from selected topic id
\t-start <id>\t start from selected topic id
\t-end <id>\t stop at selected topic id (leave this empty for infinite)
\t-noComment\t do not save comment (save data storage, bandwidth is still used)
""")

if __name__ == "__main__":
	if not os.path.exists(udg_storage_dir):
		os.makedirs(udg_storage_dir)

	index = 1
	mode = ""
	submode = {}
	## Default ##
	submode['mode'] = "get"
	mode = modeBruteID
	#############
	if len(sys.argv) > 1:
		if sys.argv[1] == '-h' or sys.argv[1] == '--help' or sys.argv[1] == "help":
			helpMode()
			exit()
		while index < len(sys.argv):
			if sys.argv[index] == "-r":
				mode = modeRoom
			elif sys.argv[index] == "-b":
				mode = modeBruteID
			elif sys.argv[index] == "-cont" or sys.argv[index] == "-c":
				submode['mode'] = "cont"
			elif sys.argv[index] == "-start":
				submode['mode'] = "start"
				index = index + 1
				submode['start'] = (int)(sys.argv[index])
			elif sys.argv[index] == "-end":
				index = index + 1
				submode['end'] = (int)(sys.argv[index])
			elif sys.argv[index] == "-tid" or sys.argv[index] == "-get":
				submode['mode'] = "get"
				index = index + 1
				submode['tid'] = (int)(sys.argv[index])
			elif sys.argv[index] == "-noComment":
				disableCommentFeature = True
			elif index == (len(sys.argv)-1) and ('start' not in submode) and ('end' not in submode):
				mode = modeBruteID
				submode['tid'] = (int)(sys.argv[index])
			else:
				print("You just typed invalid mode: " + sys.argv[index])
				print("PS: please type ID as last parameter (or after -tid, -start, -end)")
				exit()
			index = index + 1
	else:
		print("""
Please enter mode to start program.
E.g. python pantipScraper.py 35000000
For more mode: python pantipScraper.py --help
""")
		exit()

	# Arrange the input
	funcArgv = {}
	if 'end' not in submode:
		funcArgv['endID'] = 100000000000
	else:
		funcArgv['endID'] = submode['end']

	if submode['mode'] == "start":
		funcArgv['pageID'] = submode['start']
	elif submode['mode'] == "get":
		funcArgv['pageID'] = submode['tid']
		funcArgv['endID'] = submode['tid']
	mode(funcArgv)

	print("Finished scraping")


Writing pantipScraper.py


## เริ่ม scrape data

ตัวอย่าง url ที่ใช้ : https://pantip.com/topic/38952653

In [8]:
# python pantipScraper.py topic_id

!python pantipScraper.py 42069251

Success: Crawling page 42069251
Finished scraping


ไฟล์ Json จะถูก save ไว้ที่ pantip_storage

## จัดให้อยู่ในรูป Dataframe

- user_id: id ของผู้เข้ามา comment
- message: comment หลัก ( ไม่รวม comment ตอบกลับ)
- name: ชื่อกระทู้
- story: รายละเอียดที่เจ้าของกระทู้โพส


In [5]:
import pandas as pd
import json

# Read the JSON file
with open('/content/pantip_storage/ptopic42069', 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# Extract user_id and message fields
user_ids = []
messages = []
for comment in data['comments']:
    user_ids.append(comment['user_id'])
    messages.append(comment['message'])

# Create a DataFrame
df = pd.DataFrame({'user_id': user_ids, 'message': messages})

# Print the DataFrame
print(df)

    user_id                                            message
0    528154  สวัสดีค่ะ คุณสมาชิกหมายเลข 4633809<br />\n<br ...
1   2327400  แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...
2    144961  เค้าโทร.มาหาเรา​ เพราะอยากได้เงินเรา<br />\nอย...
3   4776702  การขอยกเลิกกรมธรรม์ประกันชีวิต (Free look peri...
4   1310874  เช็คกรมธรรม์ว่าเงินไขเป็นยังไง เงินคืนได้เท่าไ...
5    235041  เคยตกหลุมทำประกันทางโทรศัพท์ไป พอได้รับกรมธรรม...
6    193937  เขาไม่ให้เราอ่านเลยครับ​ น่าสงสัยจริงๆว่าทำไม<...
7   2969300  ต้องขอบคุณ จขกท. ที่ออกมาเล่าเรื่องราวแบบนี้ให...
8    291645                                       ชอบพูดไม่ครบ
9   4387777  เคยทำประกันเพื่อเอาบัตรเครดิตระดับ signature ข...
10  1509033  ทุกครั้งที่ทำประกัน ได้เล่มมาเราจะอ่านเองอีกรอ...
11   110603  จริงๆถ้าเข้าข่าย หลอกให้เข้าใจผิดในการทำประกัน...
12   674183  สายด่วน คปภ. ๑๑๘๖<br />\n<br />\nสำนักงานคณะกร...
13  7599342  ที่คุณทำน่าจะเป็นตัวเดียวกับเรานะคะ saving20/1...
14   569516  ส่วนตัวเราขี้เกียจเทียบความคุ้มค่าดอกเบี้ย

In [6]:
df['story'] = data['story']
df['name'] = data['name']
df

,user_id,message,story,name
0,528154,สวัสดีค่ะ คุณสมาชิกหมายเลข 4633809<br />\n<br ...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
1,2327400,แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
2,144961,เค้าโทร.มาหาเรา​ เพราะอยากได้เงินเรา<br />\nอย...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
3,4776702,การขอยกเลิกกรมธรรม์ประกันชีวิต (Free look peri...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
4,1310874,เช็คกรมธรรม์ว่าเงินไขเป็นยังไง เงินคืนได้เท่าไ...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
5,235041,เคยตกหลุมทำประกันทางโทรศัพท์ไป พอได้รับกรมธรรม...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
6,193937,เขาไม่ให้เราอ่านเลยครับ​ น่าสงสัยจริงๆว่าทำไม<...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
7,2969300,ต้องขอบคุณ จขกท. ที่ออกมาเล่าเรื่องราวแบบนี้ให...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
8,291645,ชอบพูดไม่ครบ,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD
9,4387777,เคยทำประกันเพื่อเอาบัตรเครดิตระดับ signature ข...,\n\t\t\t\t\t\t\t\tวันนี้จะมาแชร์ความโง่ของตัวเ...,เสียค่าโง่ให้ FWD


# ประยุกต์ใช้กับ selenium

เนื่องจากข้อมูลโพสต่างๆจะโชว์ก็ต่อเมื่อ scroll down จึงต้องใช้ selenium ในการดึงข้อมูล


[resource: วิธีการติดตั้ง selenium บน google colab](https://dataguru.cc/blog/step-by-step-guide-to-install-selenium-in-google-colab/)

In [9]:
# install selenium
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [10]:
# install chorm driver
!apt-get update
!apt-get install chromium chromium-driver

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,369 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,019 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,307 kB]
Get:13 ht

In [11]:
# import selenium library
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# import beatifulsoup and other necessary lib for web scraping
import requests
from bs4 import BeautifulSoup
import time

In [12]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


## scrape เอา file path ในหน้า Enginsearch

- กำหนดให้ scroll down  2 ครั้ง (หากต้องการข้อมูลมากกว่านี้ให้เพิ่มจำนวน scroll)

In [14]:
# Initialize the Chrome driver
driver = web_driver()

url = "https://pantip.com/search?q=FWD"
driver.get(url)

# Scroll down the page to load more content
for _ in range(2):  # Adjust the number of times to scroll
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Adjust the delay based on your network speed

# Retrieve the HTML content after scrolling
html_content = driver.page_source

# Close the driver after scraping
driver.quit()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all the div elements with the class 'pt-list-item__title'
div_list = soup.find_all('div', class_='pt-list-item__title')

# Extract the href values from the div elements
href_list = []
for div in div_list:
    if div.a and div.a['href']:
        href_list.append(div.a['href'])

# Print the extracted href values
for href in href_list:
    print(href)


https://pantip.com/topic/42419767
https://pantip.com/topic/42270835
https://pantip.com/topic/42422161
https://pantip.com/topic/42426853
https://pantip.com/topic/42633913
https://pantip.com/topic/42503626
https://pantip.com/topic/42059989
https://pantip.com/topic/42580165
https://pantip.com/topic/42621577
https://pantip.com/topic/42203642
https://pantip.com/topic/42634520
https://pantip.com/topic/42504100
https://pantip.com/topic/42609317
https://pantip.com/topic/42375738
https://pantip.com/topic/42501117
https://pantip.com/topic/42391550
https://pantip.com/topic/42252082
https://pantip.com/topic/42348098
https://pantip.com/topic/42619587
https://pantip.com/topic/42650647
https://pantip.com/topic/42460393
https://pantip.com/topic/42514893
https://pantip.com/topic/42117649
https://pantip.com/topic/42293476
https://pantip.com/topic/42442966
https://pantip.com/topic/42318110
https://pantip.com/topic/42368704
https://pantip.com/topic/42127868
https://pantip.com/topic/42246849
https://pantip

## เริ่ม scrap data ของแต่ละ file path แล้วเก็บเป็นไฟล์ Json
- show path file
- return code = 0 is success

In [15]:
import subprocess as sp

ids = [href.split('/')[-1] for href in href_list]

for id in ids:
    return_code = sp.call(['python', 'pantipScraper.py', id])
    print("Path:", f'/content/pantip_storage/{id}', "Return code:", return_code)

Path: /content/pantip_storage/42419767 Return code: 0
Path: /content/pantip_storage/42270835 Return code: 0
Path: /content/pantip_storage/42422161 Return code: 0
Path: /content/pantip_storage/42426853 Return code: 0
Path: /content/pantip_storage/42633913 Return code: 0
Path: /content/pantip_storage/42503626 Return code: 0
Path: /content/pantip_storage/42059989 Return code: 0
Path: /content/pantip_storage/42580165 Return code: 0
Path: /content/pantip_storage/42621577 Return code: 0
Path: /content/pantip_storage/42203642 Return code: 0
Path: /content/pantip_storage/42634520 Return code: 0
Path: /content/pantip_storage/42504100 Return code: 0
Path: /content/pantip_storage/42609317 Return code: 0
Path: /content/pantip_storage/42375738 Return code: 0
Path: /content/pantip_storage/42501117 Return code: 0
Path: /content/pantip_storage/42391550 Return code: 0
Path: /content/pantip_storage/42252082 Return code: 0
Path: /content/pantip_storage/42348098 Return code: 0
Path: /content/pantip_storag

In [16]:
import os

# Directory containing the files
directory = '/content/pantip_storage/'

# Common prefix of the file names
prefix = 'ptopic'

# List comprehension to construct file paths
file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.startswith(prefix)]

# Print the list of file paths
for path in file_paths:
    print(path)


/content/pantip_storage/ptopic42514
/content/pantip_storage/ptopic42375
/content/pantip_storage/ptopic42391
/content/pantip_storage/ptopic42426
/content/pantip_storage/ptopic42348
/content/pantip_storage/ptopic42306
/content/pantip_storage/ptopic42501
/content/pantip_storage/ptopic42293
/content/pantip_storage/ptopic42252
/content/pantip_storage/ptopic42059
/content/pantip_storage/ptopic42419
/content/pantip_storage/ptopic42442
/content/pantip_storage/ptopic42634
/content/pantip_storage/ptopic42203
/content/pantip_storage/ptopic42633
/content/pantip_storage/ptopic42580
/content/pantip_storage/ptopic42117
/content/pantip_storage/ptopic42504
/content/pantip_storage/ptopic42503
/content/pantip_storage/ptopic42270
/content/pantip_storage/ptopic42318
/content/pantip_storage/ptopic42127
/content/pantip_storage/ptopic42619
/content/pantip_storage/ptopic42609
/content/pantip_storage/ptopic42621
/content/pantip_storage/ptopic42368
/content/pantip_storage/ptopic42246
/content/pantip_storage/ptop

In [23]:
len(file_paths)

30

## รวมข้อมูลให้อยู่ในรูป Dataframe

In [32]:
import pandas as pd
import json

# List of file paths
file_paths

# Initialize empty lists for user_ids, messages, names, stories, and file_paths
user_ids = []
messages = []
names = []
stories = []
file_paths_list = []

# Process each JSON file
for file_path in file_paths:
    # Read the JSON file
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        data = json.load(file)

    # Extract file path
    file_paths_list.extend([file_path] * len(data['comments']))

    # Extract user_id and message fields
    for comment in data['comments']:
        user_ids.append(comment['user_id'])
        messages.append(comment['message'])

    # Extract name and story fields
    names.extend([data['name']] * len(data['comments']))
    stories.extend([data['story']] * len(data['comments']))

# Create a DataFrame
df = pd.DataFrame({'user_id': user_ids, 'message': messages, 'story': stories, 'name': names, 'file_path': file_paths_list})

# Print the DataFrame
print(df)


    user_id                                            message  \
0   2327400  แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...   
1   2327400  แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...   
2    173895  กรมธรรม์ที่เพิ่งทำมาได้ไม่นาน ปกติประกันจะขอตร...   
3    704085          พวก simple disease จะเรื่องเยอะแบบนี้แหละ   
4   7872062  ทำประกันอุบัติเหตุไว้ลื่นล้มเคลมไปตั้งแต่วันที...   
..      ...                                                ...   
90  1400377              ไม่เกินระยะหนึ่ง ต่อได้ รีบแจ้งตัวแทน   
91  6720532  ส่งเบี้ยมากี่ปีกี่เดือนแล้ว ยังไงก็รีบติดต่อตั...   
92  2327400  แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...   
93   528154  สวัสดีค่ะ คุณสมาชิกหมายเลข 7905738<br />\n<br ...   
94  1579015  ติดตั้งแอป Omne ให้จบแล้วดูรายละเอียดทั้งหมดได...   

                                                story  \
0   \n\t\t\t\t\t\t\t\tผมเป็นตัวแทน fwd นะครับ มีปั...   
1   \n\t\t\t\t\t\t\t\tเราทำประกันสุขภาพไว้กับ FWD ...   
2   \n\t\t\t\t\t\t\t\tเราทำประกันสุข

In [34]:
df.head()

,user_id,message,story,name,file_path
0,2327400,แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...,\n\t\t\t\t\t\t\t\tผมเป็นตัวแทน fwd นะครับ มีปั...,เป็นตัวแทนFwd แต่ติดต่อ eserviceไม่เคยได้,/content/pantip_storage/ptopic42514
1,2327400,แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...,\n\t\t\t\t\t\t\t\tเราทำประกันสุขภาพไว้กับ FWD ...,ทำไมเคลมประกัน FWD อยากจังเลย,/content/pantip_storage/ptopic42375
2,173895,กรมธรรม์ที่เพิ่งทำมาได้ไม่นาน ปกติประกันจะขอตร...,\n\t\t\t\t\t\t\t\tเราทำประกันสุขภาพไว้กับ FWD ...,ทำไมเคลมประกัน FWD อยากจังเลย,/content/pantip_storage/ptopic42375
3,704085,พวก simple disease จะเรื่องเยอะแบบนี้แหละ,\n\t\t\t\t\t\t\t\tเราทำประกันสุขภาพไว้กับ FWD ...,ทำไมเคลมประกัน FWD อยากจังเลย,/content/pantip_storage/ptopic42375
4,7872062,ทำประกันอุบัติเหตุไว้ลื่นล้มเคลมไปตั้งแต่วันที...,\n\t\t\t\t\t\t\t\tเราทำประกันสุขภาพไว้กับ FWD ...,ทำไมเคลมประกัน FWD อยากจังเลย,/content/pantip_storage/ptopic42375


In [35]:
df.tail()

,user_id,message,story,name,file_path
90,1400377,ไม่เกินระยะหนึ่ง ต่อได้ รีบแจ้งตัวแทน,\n\t\t\t\t\t\t\t\tใบแจ้งมาวันที่ 15 ต.ค.66 พอด...,ขาดต่อประกันชีวิต FWD ทำไงได้บ้างคะ,/content/pantip_storage/ptopic42650
91,6720532,ส่งเบี้ยมากี่ปีกี่เดือนแล้ว ยังไงก็รีบติดต่อตั...,\n\t\t\t\t\t\t\t\tใบแจ้งมาวันที่ 15 ต.ค.66 พอด...,ขาดต่อประกันชีวิต FWD ทำไงได้บ้างคะ,/content/pantip_storage/ptopic42650
92,2327400,แอดมินขออนุญาตประสานงานให้เจ้าหน้าที่ติดต่อกลั...,\n\t\t\t\t\t\t\t\tอยากยกเลิกประกันfwdทำยังไงดี...,อยากยกเลิกประกันfwd,/content/pantip_storage/ptopic42422
93,528154,สวัสดีค่ะ คุณสมาชิกหมายเลข 7905738<br />\n<br ...,\n\t\t\t\t\t\t\t\tอยากยกเลิกประกันfwdทำยังไงดี...,อยากยกเลิกประกันfwd,/content/pantip_storage/ptopic42422
94,1579015,ติดตั้งแอป Omne ให้จบแล้วดูรายละเอียดทั้งหมดได...,\n\t\t\t\t\t\t\t\tอยากยกเลิกประกันfwdทำยังไงดี...,อยากยกเลิกประกันfwd,/content/pantip_storage/ptopic42422


## Export ข้อมูล

In [39]:
df.to_csv('pantip_data.csv',encoding = 'utf8', index = False)